In [1]:
import csv
import pickle

import gensim
from gensim import models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# 오차행렬(TP, TN, FP, FN) 구하는 모듈 임포트
from sklearn.metrics import confusion_matrix

In [3]:
dataset_file_name = './dataset/corpus_myself.csv'
model_file_name = 'logistic.pkl'

In [4]:
# csv 파일로부터 X, y 데이터를 list타입으로 얻기

with open(dataset_file_name, 'r') as file:
    csv.field_size_limit(1000000)    # 필드 당 제한 길이 정하기
    data = list(csv.reader(file))    # csv 파일을 불러와 row단위로 list를 만든 후 필드를 기준(,를 기준)으로 split 함

# texts : 단어가 들어간 data (X)
# label_ids : label을 one-in-hot incoding해서 저장 (Y)
texts, label_ids = [], []
label2id = {}    # label값을 key, 여기에 들어간 순서(ID)를 value로 하는 딕셔너리
id2label = []
IDX_LABEL, IDX_SENTENCE = 0, 1
sum_words = []

for counter, row in enumerate(data):
    # if counter == 0:    #왜 첫 줄을 뺌..?
    #     continue
    label = row[IDX_LABEL]    # 해당 row의 라벨값 불러오기
    if label not in label2id:    # label이 딕셔너리에 없을 경우
        label2id[label] = len(label2id)
        id2label.append(label)
    label_ids.append(label2id[label])    # label을 one in hot encoding해서 추가
    
    # 단어 개수 합 구하기
    word_list = row[IDX_SENTENCE].split(' ')
    for word in word_list:
        sum_words.append(word)
    
    texts.append(word_list)    # 단어 단위로 분할한 데이터를 texts에 저장

print('sum_words : ', len(sum_words))
# id2label = {v: k for k, v in label2id.items()}

sum_words :  63192


In [5]:
# 텍스트와 라벨을 train과 test에 분할
X_train_texts, X_test_texts, y_train, y_test = train_test_split(texts, label_ids, test_size=0.2, random_state=42)

In [6]:
import pprint
import numpy as np
# train의 텍스트 데이터로부터 tf-idf를 계산해 해당 값으로 가중치를 부여한 행렬 작성

# 텍스트 데이터로부터 사전을 작성
dictionary = gensim.corpora.Dictionary(X_train_texts)
print(dictionary)
# 사전을 이용하여 BoW형식으로 문장을 행렬화
# (해당 토큰(단어)의 딕셔너리 ID, 해당 토큰이 현재 몇 번째 등장하는지)
# doc2bow의 파라미터에는 string을 원소로 가지는 list가 들어감
corpus = [dictionary.doc2bow(text) for text in X_train_texts]

# tf-idf 이용하여 가중치 부여
tfidf_model = models.TfidfModel(corpus)    # 모델 불러오기
tfidf_corpus = tfidf_model[corpus]    # (토큰 ID, 가중치)
print(corpus[0])
pprint.pprint(tfidf_corpus[0])

'''
# 다른 문서와 유사도 구하기
from gensim import similarities
index = similarities.SparseMatrixSimilarity(tfidf_corpus, num_features=13280)    # 유사도 행렬 생성
query_document = '코딩이 너무 하기 싫어'.split(' ')
query_bow = dictionary.doc2bow(query_document)
sims = index[tfidf_model[query_bow]]
print(list(enumerate(sims)))    # (딕셔너리 ID, 유사도)
'''

num_words = len(dictionary)
num_sentences = len(X_train_texts)
# BoW 형태의 데이터를 dense matrix로 바꾸어줌 (근데 sparse matrix아닌가..?)
# 문서 당 하나의 열로 나타나므로 전치시켜주자. (.T)
X_train_tfidf = gensim.matutils.corpus2dense(tfidf_corpus, num_terms=num_words, num_docs=num_sentences).T
print(X_train_tfidf[0])

Dictionary(13280 unique tokens: ['광물', '나침반', '띤', '막대', '올메크']...)
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(0, 0.31731451891427054),
 (1, 0.32318201730167934),
 (2, 0.39224187709400915),
 (3, 0.26641977830938135),
 (4, 0.3375132330560574),
 (5, 0.39224187709400915),
 (6, 0.4267718069901741),
 (7, 0.3465957926497777)]
[0.3173145  0.32318202 0.39224187 ... 0.         0.         0.        ]


In [7]:
# test의 텍스트 데이터에도 위와 똑같이 수행 (tf-idf 가중치 행렬 작성)
# 기존의 train 데이터로 만든 사전 이용하여 text data를 BoW행렬로 만듦
corpus_test = [dictionary.doc2bow(text) for text in X_test_texts]
tfidf_corpus_test = tfidf_model[corpus_test]
num_sentences_test = len(X_test_texts)

X_test_tfidf = gensim.matutils.corpus2dense(tfidf_corpus_test, num_terms=num_words, num_docs=num_sentences_test).T

In [8]:
# train 데이터를 이용하여 분류기 구축
# 이 때 로지스틱 회귀를 사용

clf = LogisticRegression(multi_class='multinomial', solver='lbfgs')    # C=1.0, penalty='l2' (default)
clf.fit(X_train_tfidf, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [9]:
# test 데이터 이용해 분류기의 정밀도 평가 (accuracy, precision, recall, f1-score)
y_pred = clf.predict(X_test_tfidf)
target_names = id2label

print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred))

# Micro average : averaging the total true positives, false negatives and false positives
# macro average : averaging the unweighted mean per label
# weighted average : averaging the support-weighted mean per label

              precision    recall  f1-score   support

     economy       0.89      0.85      0.87       433
  humanities       0.84      0.85      0.85       447
     science       0.82      0.86      0.84       434

   micro avg       0.85      0.85      0.85      1314
   macro avg       0.85      0.85      0.85      1314
weighted avg       0.85      0.85      0.85      1314

[[367  25  41]
 [ 28 379  40]
 [ 17  45 372]]


In [ ]:
# 이번에는 statsmodel로 Logistic Regression을 해보자.
import statsmodels.api as sm

X_train_tfidf_const = sm.add_constant(X_train_tfidf)
clf2 = sm.MNLogit(y_train, X_train_tfidf_const, check_rank=False)
result = clf2.fit()
print(result.summary())